<mark> **Please make a copy of this notebook before running it** </mark>

#Introduction: What is Cardinal?

Cardinal is a Python library, created by the [Open Contracting Partnership](https://www.open-contracting.org/) to measure red flags and procurement indicators using OCDS data.

Currently, Cardinal supports the calculation of the following red flags:

* The percentage difference between the winning bid and the second-lowest valid bid is a low outlier [(R024)](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/024.html)
* The ratio of winning bids to submitted bids for a top tenderer is a low outlier [(R025)](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/025.html)
* Bids are disqualified if not submitted by the single tenderer of the winning bid [(R035)](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/035.html)
* The lowest submitted bid is disqualified, while the award criterion is price only [(R036)](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/036.html)
* The ratio of disqualified bids to submitted bids is a high outlier per buyer, procuring entity or tenderer [(R038)](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/038.html)


To read the full documentation you can visit the [documentation site](https://cardinal.readthedocs.io/en/latest/).

In this tutorial we explain how to run Cardinal in a Google Colaboratory notebook.  We will cover the following steps:

1. Setup
2. Collecting data from the [OCDS Data Registry](https://data.open-contracting.org/)
3. Preparting the data
4. Exploring the data
5. Calculating the indicators


If you have any questions running this notebook you can write to **data@open-contracting.org**

# Setup

Install [OCDS Cardinal](https://github.com/open-contracting/cardinal-rs/blob/main/README.md) and other libraries.

In [ ]:
! curl -sSOL https://github.com/open-contracting/cardinal-rs/releases/download/0.0.5/ocdscardinal-0.0.5-linux-64-bit.zip

In [ ]:
! unzip -oj ocdscardinal-0.0.5-linux-64-bit.zip ocdscardinal-0.0.5-linux-64-bit/ocdscardinal
! ls

Archive:  ocdscardinal-0.0.5-linux-64-bit.zip
  inflating: ocdscardinal            
issues.csv    ocdscardinal-0.0.5-linux-64-bit.zip  prepared.jsonl      results.json  settings.ini
ocdscardinal  paraguay_dncp_records_2023.jsonl	   result_fields.json  sample_data


In [ ]:
import pandas as pd
from google.colab import data_table
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


# Download the data from the Data Registry

To select the data source go to the [OCDS data registry](https://data.open-contracting.org/) and select the desired publisher.  For the publisher of choice select a **jsonl file** and copy the url, in the command below.

**In the registry you will also find a description of the data source and direct links to the publisher website where you can find more information about the scope of the publication.**

<img src="https://drive.google.com/uc?id=10dlm8c55pN89YTGEyZgvsLDc8fFMLNf0"  width="200" height="300">

In this example we will use data from the Dominican Republic.  

In [ ]:
#url = input('Add URL from the registry source:')

In [ ]:
url="https://data.open-contracting.org/en/publication/63/download?name=2023.jsonl.gz"

In [ ]:
! curl -sSOJ "$url"

curl: (23) Failed writing header


In the files tab at the left hand side of the notebook, look for the file ending in .gz you just downloaded (e.g `chile_compra_api_releases_full.jsonl.gz`), and add it to the command below (see example):

<img src="https://drive.google.com/uc?id=19z86Nj5OY7Y8REfcd2sZbFPXDTAWZYS6" width="200" height="200">


In [ ]:
#file=input('Add name of .gz file: ')

In [ ]:
file="/content/paraguay_dncp_records_2023.jsonl.gz"

In [ ]:
file_jsonl=file.replace('.gz', '')

In [ ]:
! gunzip -f $file

In [ ]:
! ls -lh $file_jsonl

-rw-r--r-- 1 root root 1.6G Jan 16 16:38 /content/paraguay_dncp_records_2023.jsonl


# Prepare the data

###See which fields are published

Before you start calculating any indicators you can use the [`coverage`](https://cardinal.readthedocs.io/en/latest/cli/coverage.html) command, to see which fields are published in the dataset.

In [ ]:
! ./ocdscardinal coverage  $file_jsonl >> result_fields.json

In [ ]:
fields

In [66]:
fields = pd.DataFrame(pd.read_json('result_fields.json', typ='series', lines=True), columns=['releases']).rename_axis('path').reset_index()
#Leaves only object members
fields=fields[fields.path.str.contains('[a-z]$')]
fields['path']=fields['path'].str.replace(r'[][]|^/', '', regex=True)
fields_list=fields['path'].tolist()
fields

AttributeError: Can only use .str accessor with string values!

### Write a default settings file

Use the [`init`](https://cardinal.readthedocs.io/en/latest/cli/init.html) command to write a default settings file for configuration.

In [68]:
! ./ocdscardinal init settings.ini

Settings written to "settings.ini".


Preview the content of the settings file. You can edit the [settings file](https://cardinal.readthedocs.io/en/latest/topics/settings.html) as needed, to configure the behavior of the `prepare` and `indicators` commands.  As a default, the settings file enables all indicators.

In [69]:
! ./ocdscardinal init -

; currency = USD

; `prepare` command
;
; Read the documentation at:
; https://cardinal.readthedocs.io/en/latest/cli/prepare.html

[defaults]
; currency = USD
; item_classification_scheme = UNSPSC
; bid_status = valid
; award_status = active

[codelists.BidStatus]
; qualified = valid

[codelists.AwardStatus]
; Active = active

; `indicators` command
;
; Read the documentation at:
; https://cardinal.readthedocs.io/en/latest/cli/indicators/

[R024]
; threshold = 0.05

[R025]
; percentile = 75
; threshold = 0.05

[R035]
; threshold = 1

[R036]

[R038]
; threshold = 0.5


### Identify and review data quality issues

You can use the [`prepare`](https://cardinal.readthedocs.io/en/latest/cli/prepare.html) command to identify if the dataset has data quality issues and correct them before calculating the indicators.

In [70]:
! ./ocdscardinal prepare --settings settings.ini --output prepared.jsonl --errors issues.csv $file_jsonl

Check the issues reported:

In [71]:
! wc issues.csv

  905  1810 49846 issues.csv


In [72]:
! head -n 10 issues.csv

7326,ocds-03ad3f-430176,/awards[]/status,8,,not set
10092,ocds-03ad3f-422116,/awards[]/status,25,,not set
10092,ocds-03ad3f-422116,/awards[]/status,26,,not set
10092,ocds-03ad3f-422116,/awards[]/status,27,,not set
10092,ocds-03ad3f-422116,/awards[]/status,28,,not set
10092,ocds-03ad3f-422116,/awards[]/status,33,,not set
10092,ocds-03ad3f-422116,/awards[]/status,34,,not set
10092,ocds-03ad3f-422116,/awards[]/status,35,,not set
10091,ocds-03ad3f-422114,/awards[]/status,36,,not set
10091,ocds-03ad3f-422114,/awards[]/status,37,,not set


Read issues into a data frame:

In [73]:
import pandas as pd

df = pd.read_csv("issues.csv", header=None)
df.columns = ["line", "ocid", "path", "array indexes", "incorrect value", "error description"]
df.head()

,line,ocid,path,array indexes,incorrect value,error description
0,7326,ocds-03ad3f-430176,/awards[]/status,8,NaN,not set
1,10092,ocds-03ad3f-422116,/awards[]/status,25,NaN,not set
2,10092,ocds-03ad3f-422116,/awards[]/status,26,NaN,not set
3,10092,ocds-03ad3f-422116,/awards[]/status,27,NaN,not set
4,10092,ocds-03ad3f-422116,/awards[]/status,28,NaN,not set


Check which errors occur per path.  In this case the dataset does not have a classification scheme set and has invalid values in the bids status and some entries without the value set.  

In [74]:
df[["path", "error description"]].value_counts().reset_index()

,path,error description,0
0,/awards[]/status,not set,900
1,/bids/details[]/status,not set,5


Check the invalid values in the bids status:

In [75]:
df[(df['error description']=='invalid')].groupby('incorrect value').size().reset_index()

,incorrect value,0


You can correct the data quality issues in the settings file.  You can [read in the documentation](https://cardinal.readthedocs.io/en/latest/cli/prepare.html#configuration) how to:


* Normalize ID fields
* Fill in missing values
* Re-map incorrect codes

For example, we can set the value in the settings file:


```
[defaults]
item_classification_scheme = UNSPSC
```


Once you re-write the settings file with the new configurations, you can re-run the prepare command.  In this example we will set the award status.

In [76]:
%%writefile settings.ini
[defaults]
item_classification_scheme = UNSPSC

[codelists.BidStatus]
Qualified = valid
Disqualified = disqualified
InTreatment = pending

[codelists.AwardStatus]
; Active = active

; `indicators` command
;
;/content/paraguay_hacienda_full.jsonl.gz Read the documentation at:
; https://cardinal.readthedocs.io/en/latest/cli/indicators/

[R024]
; threshold = 0.5

[R025]
; percentile = 75
; threshold = 0.05

[R035]
; threshold = 1

[R036]

[R038]
minimum_submitted_bids = 100

Overwriting settings.ini


Re-run the prepare command with the new settings.

In [77]:
! ./ocdscardinal prepare --settings settings.ini --output prepared.jsonl --errors issues.csv $file_jsonl

# Calculate indicators

You can calculate the indicators using the [`indicators`](https://cardinal.readthedocs.io/en/latest/cli/indicators/index.html) command.  The command will calculate all the indicators enabled in the settings file.

The command below writes the output in `results.json`

In [78]:
! ./ocdscardinal indicators -s settings.ini prepared.jsonl > results.json

Check the number of results:

In [79]:
! wc -c results.json

4811 results.json


### Review the output

You can read in detail in [the documentation](https://cardinal.readthedocs.io/en/latest/cli/indicators/index.html#demonstration) how to interpret the output.

The JSON output is organized as an object in which the key is a group: “OCID” (the unique identifier of a contracting process), “Buyer”, “ProcuringEntity” or “Tenderer”.


```
{
  "OCID": {},
  "Buyer": {},
  "ProcuringEntity": {},
  "Tenderer": {}
}
```
Each value at the top level is an object representing the results within that group, in which the key is an identifier extracted from the input data.  Each value at the second level is an object representing the results relating to that identifier, in which the key is the code for an indicator, and the value is the output of that indicator.

```
{
  "OCID": {
    "ocds-6550wx-JRFPFA-DAF-CM-2021-0012": {
      "R036": 1.0
    }
  },
  "Buyer": {
    "DO-RPE-55216": {
      "R038": 0.8
    }
  }
}
```
The JSON output also has a Meta key at the top level. Its value is an object with information about the quartiles and fences used to calculate the results, rounded to 4 decimals.

```
{
  // ...
  "Meta": {
    "R024": {
      "q1": 66.6667,
      "q3": 100.0,
      "lower_fence": 16.6667
    }
  }
}
```

The output below shows the values of the configurations of the indicators in the Meta key.

In [80]:
import json
from pprint import pprint
with open("results.json") as f:
    data = json.load(f)
data['Meta']

{'R025': {'q1': 0.0, 'q3': 1.0, 'upper_fence': 3.0, 'lower_fence': -1.5},
 'R038': {'Buyer_q1': 0.0,
  'Buyer_q3': 0.0,
  'Buyer_upper_fence': 0.0,
  'ProcuringEntity_q1': 0.0,
  'ProcuringEntity_q3': 0.0,
  'ProcuringEntity_upper_fence': 0.0},
 'R024': {'q1': None, 'q3': None, 'lower_fence': None}}

In the data variable the results are stored in a dictionary.  Since the flags can be calculated by contracting process (ocid), tenderer or procuring entity, we can explore the results for each.  

In [81]:
data

{'Buyer': {'DNCP-SICP-CODE-178': {'R038': 0.0},
  'DNCP-SICP-CODE-157': {'R038': 0.0},
  'DNCP-SICP-CODE-106': {'R038': 0.0},
  'DNCP-SICP-CODE-1': {'R038': 0.0},
  'DNCP-SICP-CODE-41': {'R038': 0.0},
  'DNCP-SICP-CODE-335': {'R038': 0.0},
  'DNCP-SICP-CODE-305': {'R038': 0.0},
  'DNCP-SICP-CODE-189': {'R038': 0.0},
  'DNCP-SICP-CODE-137': {'R038': 0.0},
  'DNCP-SICP-CODE-30': {'R038': 0.0},
  'DNCP-SICP-CODE-191': {'R038': 0.0},
  'DNCP-SICP-CODE-24': {'R038': 0.0},
  'DNCP-SICP-CODE-304': {'R038': 0.0},
  'DNCP-SICP-CODE-307': {'R038': 0.0},
  'DNCP-SICP-CODE-100': {'R038': 0.0},
  'DNCP-SICP-CODE-82': {'R038': 0.0},
  'DNCP-SICP-CODE-169': {'R038': 0.0},
  'DNCP-SICP-CODE-300': {'R038': 0.0},
  'DNCP-SICP-CODE-299': {'R038': 0.0},
  'DNCP-SICP-CODE-317': {'R038': 0.0},
  'DNCP-SICP-CODE-320': {'R038': 0.0},
  'DNCP-SICP-CODE-303': {'R038': 0.0},
  'DNCP-SICP-CODE-25': {'R038': 0.0},
  'DNCP-SICP-CODE-11': {'R038': 0.0},
  'DNCP-SICP-CODE-21': {'R038': 0.0},
  'DNCP-SICP-CODE-337': {

####Explore flags by ocid.

The flags [R036](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/036.html) (lowest bid disqualified) and [R035](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/035.html) (All except winning bid disqualified) were calculated.  

In [82]:
ocid = pd.DataFrame.from_dict(data["OCID"], orient="index").reset_index()
ocid.head()

KeyError: 'OCID'

29121 procedures were flagged for disqualifying the lowest bid when the award criteria was price only.  

In [83]:
len(ocid)

NameError: name 'ocid' is not defined

#### Explore flags by tenderer

The flag [R038](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/038.html) (Excessive disqualified bids) was calculated.

In [87]:
tenderer = pd.DataFrame.from_dict(data["Buyer"], orient="index").reset_index()
tenderer.head()

,index,R038
0,DNCP-SICP-CODE-1,0.0
1,DNCP-SICP-CODE-100,0.0
2,DNCP-SICP-CODE-104,0.0
3,DNCP-SICP-CODE-105,0.0
4,DNCP-SICP-CODE-106,0.0


Count the number of flags per tenderer

In [88]:
tenderer_flags=pd.melt(tenderer, id_vars=['index'], value_vars=['R038'])

tenderer_flags[~tenderer_flags.value.isna()].groupby('index')\
.agg(total_flags=('variable', 'count')).sort_values(by='total_flags', ascending=False).reset_index()

,index,total_flags
0,DNCP-SICP-CODE-1,1
1,DNCP-SICP-CODE-335,1
2,DNCP-SICP-CODE-303,1
3,DNCP-SICP-CODE-304,1
4,DNCP-SICP-CODE-305,1
5,DNCP-SICP-CODE-306,1
6,DNCP-SICP-CODE-307,1
7,DNCP-SICP-CODE-308,1
8,DNCP-SICP-CODE-309,1
9,DNCP-SICP-CODE-31,1


There are 292 tenderers with all their bids disqualified.  

In [89]:
import plotly.express as px
fig = px.histogram(tenderer, x="R038", marginal = 'box', template='plotly_white',
                   title="R038 distribution tenderers")
fig.update_traces(marker_color='#D6E100')
fig.update_layout(width=600, height=400)
fig.show()

In [90]:
#Filter tenderers with all their bids diqualified
R038_tenderer=tenderer[tenderer['R038']==1]

You can explore the results to see if they make sense.  To explore the data we will use [flatterer](https://flatterer.opendata.coop/) to flatten the json into csv tables.  

In [91]:
!pip install flatterer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.1/22.1 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 11.9 MB/s eta 0:00:00


In [ ]:
import flatterer
output = flatterer.flatten('dominican_republic_api_full.jsonl', 'dom', json_stream=True)

Most of the bidders with all their bids disqualified had submitted 2 bids.  You can decide to adjust the threshold

In [ ]:
#Load the bid tenderers table.
bids_tenderers=pd.read_csv('/content/dom/csv/bids_details_tenderers.csv')

#Filter only flagged bidders with a 100% of bids disqualified, and calculate the number of bids.
bidders_distribution=bids_tenderers[bids_tenderers['id'].isin(R038_tenderer['index'])]\
.groupby('id').agg(total=('id', 'count')).reset_index()

#Plot the distribution.
fig = px.histogram(bidders_distribution, x="total", marginal = 'box', template='plotly_white',
                   title="Number of bids by flagged bidders (R038)")
fig.update_traces(marker_color='#D6E100')
fig.update_layout(width=500, height=400)
fig.show()

#### Explore flags by procuring entity

In [ ]:
procuring_entity = pd.DataFrame.from_dict(data["ProcuringEntity"], orient="index").reset_index()
procuring_entity.head()

In [ ]:
len(procuring_entity)

In [ ]:
fig = px.histogram(procuring_entity, x="R038", marginal = 'box', template='plotly_white',
                   title="R038 distribution procuring entity")
fig.update_traces(marker_color='#D6E100')
fig.update_layout(width=600, height=400)
fig.show()

Check the number of procedures of the flagged entities.

In [ ]:
#Load the tenders (main) table.
tenders=pd.read_csv('/content/dom/csv/main.csv')

In [ ]:
entities_distribution=tenders[tenders['tender_procuringEntity_id'].astype(str).isin(procuring_entity['index'])]\
.groupby('tender_procuringEntity_id').agg(total=('tender_procuringEntity_id', 'count')).reset_index()

fig = px.histogram(entities_distribution, x="total", marginal = 'box', template='plotly_white',
                   title="Number of bids by flagged bidders (R038)")
fig.update_traces(marker_color='#D6E100')
fig.update_layout(width=500, height=400)
fig.show()

The entities flagged have very few procedures, so we can check the distribution of the number of procedures by entity for the full dataset, to see if we should adjust the threshold.  

The results show that Q1=6, so we could use this threshold `minimum_contracting_processes=6` in the settings file and re-run the indicators.

In [ ]:
entities_distribution1=tenders.groupby('tender_procuringEntity_id')\
.agg(total=('tender_procuringEntity_id', 'count')).reset_index()

fig = px.histogram(entities_distribution1, x="total", marginal = 'box', template='plotly_white',
                   title="Number of bids by flagged bidders (R038)")
fig.update_traces(marker_color='#D6E100')
fig.update_layout(width=500, height=400)
fig.show()